# Tutorial for WorkSpace

`Workspace` is designed for helping developers manage their data and models.
This tool can simulate a virtual os-like system to operate space nodes,
which can be pointed to a file-system node or a workspace node.

## Import workspace

In [1]:
from longling.ml.workspace import workspace as ws

## Create a workspace

In [2]:
wsm = ws.use("ws")
wsm.root

F:\Program\Python\longling\longling\ML\Workspace\workspace.py:154: UserWarning: WorkSpace ws not existed, created, configuration is located at ~\.lws\ws (abspath: F:\Program\Python\longling\examples\ML\~\.lws\ws). To avoid this warning, use init() to create space before using.
  % (name, wsm.config_path, os.path.abspath(wsm.config_path))


Space[ws - (a)]: / -> /

Here we create a workspace with a workspace manager, named as `ws`, this workspace pointing to nothing.
To point to a file-system location, we can firstly access the workspace `root` and
use `point` method to set the `pointer` of `root`:

In [3]:
wsm.root.point_to("./data")
wsm.root

Space[ws - (f)]: / -> ./data

## Operate workspace
Similar to file system, workspace provides developers with various file-system-like operations:

In [4]:
# create a `data` space under `root`
wsm.mkdir("data")
# create a `model` space under `root`, `mkn` is equivalent to `mkdir`
wsm.mkn("model")
# recursive create a space by path
wsm.mkn("space/model1", recursive=True)
# show all created spaces
wsm.la()

(f) / -> ./data
(a) \data -> data\data
(a) \model -> data\model
(a) \space -> data\space
(a) \space\model1 -> data\space\model1


In [5]:
# show `space` space only
wsm.ls("space")

model1


In [6]:
# show more information about `space`
wsm.ll("space")

Space[space - (a)]: \space -> data\space
----------------------------------------
model1 -> data\space\model1
----------------------------------------


From the displayed information, we can see that, a space path points to a file-system path.
For instance, `\model` is a space path and points to the file-system location `data/model`.
Developers could use `.sp` and `.fp` to access the space path and file-system path:

In [7]:
wsm["space/model1"].sp

PureWindowsPath('/space/model1')

In [8]:
wsm["space/model1"].fp

PureWindowsPath('data/space/model1')

Here, we can see three spaces under `/`, i.e., `data`, `model` and `space`.
And a space `model1` created under `space`.
Furthermore, we can see there are a little differences at the beginning of spaces.
`/` is marked as `(f)` while others are marked as `(a)`. `(f)` in fact means `file-system pointer`
while `(a)` is the `adaptive` pointer. `(f)` type means this node has been bind with the `file-system` node
and no matter how the space path changed, the pointer will point to the same `file-system` location.
On the contrary, the `(a)` type means the pointer is dynamic and adaptively changed according to the space path.
For example,

In [9]:
wsm.mkn("space/data", pointer="./data/data")
wsm["space/data"]

Space[data - (f)]: \space\data -> ./data/data

By default, if a space has not been set to be pointed to any `file-system` location,
i.e., the `pointer` of the space is `None`, it will be `(a)` type.
To see the type of a specific node, use `.ntype` method.


In [10]:
wsm["space/model1"].ntype


'a'

In [11]:
wsm["space/data"].ntype


'f'

In addition to `a` and `f`, there is another type `s`, means a space link. To create a `s` space,
just set the pointer pointing to a space, which can be easily implemented by `mkl`.
`mkl` can also be used to make a space `f`, where the pointer is expected to be a `PATH_TYPE`(e.g., `str`) variable.
Similarly, to make a space `a` type, set the pointer as `None`, which can be realized by `reset_pointer`.

In [12]:
wsm.mkl("space/data", "../data")
wsm["space/data"]

Space[data - (f)]: \space\data -> ../data

In [13]:
wsm["space/model1"]

Space[model1 - (a)]: \space\model1 -> data\space\model1

In [14]:
# move the space, to see how the file pointer changed
wsm.mv("space/model1", "space/model2")
wsm["space/model2"]

Space[model2 - (a)]: \space\model2 -> data\space\model2

In [15]:
wsm.mkn("space/model3", "./data/space/model3")
wsm["space/model3"]

Space[model3 - (f)]: \space\model3 -> ./data/space/model3

In [16]:
wsm.mv("space/model3", "space/model4")
wsm["space/model4"]

Space[model4 - (f)]: \space\model4 -> ./data/space/model3

In [17]:
wsm["space/data"].reset_pointer()
wsm["space/data"]

Space[data - (a)]: \space\data -> data\space\data

In [18]:
wsm.mkl("space/data", wsm["data"])
wsm["space/data"]

Space[data - (s)]: \space\data -> data\data

Another one method to make a space `f` is `mount`. Different from `mkl`, `mount` only accepts the `PATH_TYPE`
as the pointer argument.
*Notice: if pointer is not set, it will automatically use current space path as the file-system path*

In [19]:
wsm.mount("space/data", "./data")
wsm["space/data"]

Space[data - (f)]: \space\data -> ./data

In [20]:
wsm["space/data"].reset_pointer()
wsm["space/data"]

Space[data - (a)]: \space\data -> data\space\data

In [21]:
wsm.mount("space/data")
wsm["space/data"]

Space[data - (f)]: \space\data -> data\space\data

The other operations like `cp` and `rm` are also supported.

In [22]:
wsm.cp("space", "new_space")
wsm.la()

(f) / -> ./data
(a) \data -> data\data
(a) \model -> data\model
(a) \space -> data\space
(f) \space\data -> data\space\data
(a) \space\model2 -> data\space\model2
(f) \space\model4 -> ./data/space/model3
(a) \new_space -> data\new_space
(f) \new_space\data -> data\space\data
(a) \new_space\model2 -> data\new_space\model2
(f) \new_space\model4 -> ./data/space/model3


In [23]:
wsm.rm("new_space", recursive=True)
wsm.la()

(f) / -> ./data
(a) \data -> data\data
(a) \model -> data\model
(a) \space -> data\space
(f) \space\data -> data\space\data
(a) \space\model2 -> data\space\model2
(f) \space\model4 -> ./data/space/model3


Another one basic but important feature in workspace is make index, i.e., `mki`,
which can help developer easily access a deep-path space.

In [24]:
wsm.mkn("space/5/4/3/2/1", recursive=True)
wsm.mki("space/5/4/3/2/1", "space1")
wsm.index["space1"]

Space[1 - (a)]: \space\5\4\3\2\1 -> data\space\5\4\3\2\1

In [25]:
# above is equivalent to the following:
wsm.mkn("space/5/4/3/2/2", recursive=True, index="space2")
wsm.index["space2"]


Space[2 - (a)]: \space\5\4\3\2\2 -> data\space\5\4\3\2\2

## Advanced Operations

To build the workspace more efficient, we provide two important features: `from_fs` and `create_workspace` from template.
The former feature can generate a space based on current `file-system` structure,
while `create_workspace` can quickly generate a complicated space structure based on pre-defined template.


In [26]:
wsm = ws.use("ws", rfp="../static/ws_demo_fs", space_dir="./", force_reinit=True, from_fs=True, skip_files=False, file_as_f=True)
wsm.mkdir("model")
wsm.add_space_to(ws.create_workspace("model1", "simple"), "model", index="model1")
wsm.la()

(f) / -> ../static/ws_demo_fs
(a) \dataset1 -> ..\static\ws_demo_fs\dataset1
(a) \dataset1\cv -> ..\static\ws_demo_fs\dataset1\cv
(a) \dataset1\cv\0 -> ..\static\ws_demo_fs\dataset1\cv\0
(f) \dataset1\cv\0\train.csv -> dataset1\cv\0\train.csv
(f) \dataset1\train.csv -> dataset1\train.csv
(a) \model -> ..\static\ws_demo_fs\model
(a) \model\model1 -> ..\static\ws_demo_fs\model\model1
(a) \model\model1\input -> ..\static\ws_demo_fs\model\model1\input
(a) \model\model1\output -> ..\static\ws_demo_fs\model\model1\output
(a) \model\model1\model -> ..\static\ws_demo_fs\model\model1\model
(a) \model\model1\tmp -> ..\static\ws_demo_fs\model\model1\tmp


F:\Program\Python\longling\longling\ML\Workspace\workspace.py:154: UserWarning: WorkSpace ws not existed, created, configuration is located at ws (abspath: F:\Program\Python\longling\examples\ML\ws). To avoid this warning, use init() to create space before using.
  % (name, wsm.config_path, os.path.abspath(wsm.config_path))


## Load and Save

Workspace can be used across different scripts by `save & load`

In [27]:
wsm.save()
wsm.config_path

PureWindowsPath('ws')

In [28]:
wsm = ws.use("ws", space_dir="./")
wsm

WorkSpace[ws]

In [29]:
# A more easy way to to use `use_space`

with ws.use_space("ws1", space_dir="./", force_reinit=True) as wsm:
    wsm.mkn("space/5/3", recursive=True)
    wsm.la()

(a) / -> /
(a) \space -> \space
(a) \space\5 -> \space\5
(a) \space\5\3 -> \space\5\3


F:\Program\Python\longling\longling\ML\Workspace\workspace.py:154: UserWarning: WorkSpace ws1 not existed, created, configuration is located at ws1 (abspath: F:\Program\Python\longling\examples\ML\ws1). To avoid this warning, use init() to create space before using.
  % (name, wsm.config_path, os.path.abspath(wsm.config_path))


In [30]:
wsm = ws.use("ws1", space_dir="./")
wsm

WorkSpace[ws1]

In [31]:
wsm.la()

(a) / -> /
(a) \space -> \space
(a) \space\5 -> \space\5
(a) \space\5\3 -> \space\5\3
